# 9.2 Types of Joins

In the previous lesson, we saw how to _merge_ (or _join_) two data sets by matching on certain variables. But what happens when a row in one `DataFrame` has no match in the other? 

First, let's investigate how _pandas_ handles this situation by default. The name "Nevaeh", which is "heaven" spelled backwards, took after Sonny Sandoval of the band P.O.D. gave his daughter the name in 2000. Let's look at how common this name was five years earlier and five years after.

In [1]:
import pandas as pd

data_dir = "http://dlsun.github.io/pods/data/names/"

names1995 = pd.read_csv(data_dir + "yob1995.txt",
                        header=None, names=["Name", "Sex", "Count"])
names2005 = pd.read_csv(data_dir + "yob2005.txt",
                        header=None, names=["Name", "Sex", "Count"])

In [2]:
names1995[names1995.Name == "Nevaeh"]

,Name,Sex,Count


In [3]:
names2005[names2005.Name == "Nevaeh"]

,Name,Sex,Count
68,Nevaeh,F,4552
21353,Nevaeh,M,56


In 1995, there were no girls (at least fewer than 5) named Nevaeh; just eight years later, there were over 4500 girls (and even 56 boys) with the name. It seems like Sonny Sandoval had a huge effect.

What happens to the name "Nevaeh" when we merge the two data sets?

In [4]:
names = names1995.merge(names2005, on=["Name", "Sex"])
names[names.Name == "Nevaeh"]

,Name,Sex,Count_x,Count_y


By default, _pandas_ only includes combinations that are present in _both_ `DataFrame`s. If it cannot find a match for a row in one `DataFrame`, then the combination is simply dropped.

But in this context, the fact that a name does not appear in one data set is informative. It means that no babies were born in that year with that name. We might want to include names that appeared in only one of the two `DataFrame`s, rather than just the names that appeared in both. 

There are four types of joins, distinguished by whether they include the rows from the left `DataFrame`, the right `DataFrame`, both, or neither:

1. _inner join_ (default): only values that are present in both `DataFrame`s are included in the result
2. _outer join_: any value that appears in either `DataFrame` is included in the result
3. _left join_: any value that appears in the left `DataFrame` is included in the result, whether or not it appears in the right `DataFrame`
4. _right join_: any value that appears in the right `DataFrame` is included in the result, whether or not it appears in the left `DataFrame`.

One way to visualize the different types of joins is using Venn diagrams. The shaded region indicates which rows that are included in the output. For example, only rows that appear in both the left and right `DataFrame`s are included in the output of an inner join.

![](https://github.com/dlsun/pods/blob/master/09-Joining-Tabular-Data/joins.png?raw=1)

In _pandas_, the join type is specified using the `how=` argument.

Now let's look at examples of each of these types of joins.

In [5]:
# inner join
names_inner = names1995.merge(names2005, on=["Name", "Sex"], how="inner")
names_inner

,Name,Sex,Count_x,Count_y
0,Jessica,F,27935,8108
1,Ashley,F,26603,13270
2,Emily,F,24378,23930
3,Samantha,F,21646,13633
4,Sarah,F,21369,11527
...,...,...,...,...
19119,Zeppelin,M,5,7
19120,Zerek,M,5,8
19121,Zhen,M,5,7
19122,Ziggy,M,5,6


In [6]:
# outer join
names_outer = names1995.merge(names2005, on=["Name", "Sex"], how="outer")
names_outer

,Name,Sex,Count_x,Count_y
0,Jessica,F,27935.0,8108.0
1,Ashley,F,26603.0,13270.0
2,Emily,F,24378.0,23930.0
3,Samantha,F,21646.0,13633.0
4,Sarah,F,21369.0,11527.0
...,...,...,...,...
39490,Zymiere,M,NaN,5.0
39491,Zyrell,M,NaN,5.0
39492,Zyrian,M,NaN,5.0
39493,Zyshon,M,NaN,5.0


Names like "Zyrell" and "Zyron" appeared in the 2005 data but not the 1995 data. For this reason, their count in 1995 is `NaN`. In general, there will be missing values in `DataFrame`s that result from an outer join. Any time a value appears in one `DataFrame` but not the other, there will be `NaN`s in the columns from the `DataFrame` missing that value.

In [7]:
names_outer.isnull().sum()

Name           0
Sex            0
Count_x    13415
Count_y     6956
dtype: int64

By contrast, there are no `NaN`s when we do an inner join. That is because we restrict to only **Name** and **Sex** pairs that appeared in both `DataFrame`s, which guarantees that there are counts for both 1995 and 2005.

In [8]:
names_inner.isnull().sum()

Name       0
Sex        0
Count_x    0
Count_y    0
dtype: int64

Left and right joins preserve data from one `DataFrame` but not the other. For example, if we were trying to calculate the percentage change for each name from 1995 to 2005, we would want to include all of the names that appeared in the 1995 data. If the name did not appear in the 2005 data, then that is informative.

In [9]:
# left join
names_left = names1995.merge(names2005, on=["Name", "Sex"], how="left")
names_left

,Name,Sex,Count_x,Count_y
0,Jessica,F,27935,8108.0
1,Ashley,F,26603,13270.0
2,Emily,F,24378,23930.0
3,Samantha,F,21646,13633.0
4,Sarah,F,21369,11527.0
...,...,...,...,...
26075,Zerek,M,5,8.0
26076,Zhen,M,5,7.0
26077,Ziggy,M,5,6.0
26078,Zuberi,M,5,NaN


The result of a left join has `NaN`s in the columns from the right `DataFrame`.

In [10]:
names_left.isnull().sum()

Name          0
Sex           0
Count_x       0
Count_y    6956
dtype: int64

The result of a right join, on the other hand, has `NaN`s in the column from the left `DataFrame`.

In [11]:
# right join
names_right = names1995.merge(names2005, on=["Name", "Sex"], how="right")
names_right

,Name,Sex,Count_x,Count_y
0,Emily,F,24378.0,23930
1,Emma,F,5041.0,20335
2,Madison,F,9775.0,19562
3,Abigail,F,7821.0,15747
4,Olivia,F,7624.0,15691
...,...,...,...,...
32534,Zymiere,M,NaN,5
32535,Zyrell,M,NaN,5
32536,Zyrian,M,NaN,5
32537,Zyshon,M,NaN,5


In [12]:
names_right.isnull().sum()

Name           0
Sex            0
Count_x    13415
Count_y        0
dtype: int64

# Exercises

Exercises 1-2 deal with the Movielens data (https://dlsun.github.io/pods/data/ml-1m/ ), which contains 1 million movie ratings submitted by users. The information about the movies, ratings, and users are stored in three separate files, called `movies.dat`, `ratings.dat`, and `users.dat`. The column names are not included with the data files. Refer to the webpage above for more information.

In [53]:
df_movies = pd.read_csv("https://dlsun.github.io/pods/data/ml-1m/movies.dat", delimiter="::", encoding_errors="ignore",
                        names=["MovieID", "Title", "Genres"])
df_movies.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [54]:
df_ratings = pd.read_csv("https://dlsun.github.io/pods/data/ml-1m/ratings.dat", delimiter="::", encoding_errors="ignore",
                         names=["UserID", "MovieID", "Rating", "Timestamp"])
df_ratings.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [16]:
df_users = pd.read_csv("https://dlsun.github.io/pods/data/ml-1m/users.dat", delimiter="::", encoding_errors="ignore",
                       names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])
df_users.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


1\. Calculate the number of ratings by movie. How many of the movies had zero ratings?

(_Hint_: Why is an inner join not sufficient here?)

**An inner join is not sufficient here because movies that don't have ratings would not be included.**

In [59]:
movie_ratings = pd.merge(df_movies, df_ratings, on="MovieID", how="left")
movie_ratings.head()

,MovieID,Title,Genres,UserID,Rating,Timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,5.0,978824268.0
1,1,Toy Story (1995),Animation|Children's|Comedy,6.0,4.0,978237008.0
2,1,Toy Story (1995),Animation|Children's|Comedy,8.0,4.0,978233496.0
3,1,Toy Story (1995),Animation|Children's|Comedy,9.0,5.0,978225952.0
4,1,Toy Story (1995),Animation|Children's|Comedy,10.0,5.0,978226474.0


In [60]:
movie_ratings["Rating"].isna().sum()

177

**177 movies had zero ratings.**

2\. How many movies received both a 1 and a 5 rating? Answer this question by joining two appropriate tables.

In [63]:
oneRating = movie_ratings.loc[movie_ratings["Rating"] == 1]

In [62]:
fiveRating = movie_ratings.loc[movie_ratings["Rating"] == 5]

In [69]:
bothMovies = pd.merge(oneRating, fiveRating, on="MovieID")
len(bothMovies["MovieID"].unique())

2986

**2986 movies received both a 1 and a 5 rating.**